# Data and Model Iteration

# Data lineage
A summary of the data lineage can be found in the README.md of the [repo](https://github.com/artemiorimando/samsung-capstone).

## Image patch generation

### The data
We are working with three sqlite databases corresponding to three different datasets described in the author’s [paper](https://www.nature.com/articles/s41597-020-00756-z). The databases store the annotations and their locations for all 21 whole slide images (WSIs), but not the images themselves. The WSIs are stored in a separate (non-tracked in Git) folder because they occupy ~35 GB. A summary of the total number of mitotic and non-mitotic cells is shown below.

| Database/set  | No. of mitotic figures  | No. of non-mitotic cells  |
|---|---|---|
| MEL   | 13342  | 26526  |
| ODAEL   | 14151  | 36135  |
| CODAEL | 13907  | 36379  |

### Image patches
In order to create a cell patch classifier model we needed to generate image patches from the WSIs. The [extract_patches.ipynb](https://github.com/artemiorimando/samsung-capstone/blob/master/src/data/extract-patches.ipynb) was created to match the annotation/location data stored in the sqlite databases with the WSIs and generate 250 x 250 pixel image patches of each annotation. 250 px was chosen since the input size to many pre-trained CNNs is 224 x 224 px. 250 px enables resizing images to 224 px by downsampling.

### Test sets

We split the 21 WSIs into three groups of seven leaving one group aside as the test set in a k-fold like validation scheme.
* slidelist_test_1 = ['18','3', '22','10','15','21','14']
* slidelist_test_2 = ['1', '20','17','5', '2', '11','16']
* slidelist_test_3 = ['12','13','7', '19','8', '6', '9']
* Note: there is no slide 4, which is why there is a slide 22. There are 21 WSIs total.

[extract_patches.ipynb](https://github.com/artemiorimando/samsung-capstone/blob/master/src/data/extract-patches.ipynb) iterates through each database (MEL, ODAEL, CODAEL) and test slide list combination to generate a total of 9 image datasets. The images are stored in directories with the following structure



- Database name + test slide list number (e.g., MEL_1 or CODAEL_3)
	- train
		- Mitosis
		- Nonmitosis
	- test
		- Mitosis
		- Nonmitosis

This structure makes it easy to load datasets into common deep learning frameworks like PyTorch and Tensorflow.

# Cell patch classifier model with PyTorch

    Goal: train a CNN model in PyTorch to classify image patches as mitotic or non-mitotic

We wanted to replicate the author's work in a common deep learning framework instead of FastAI. So we implemented a transfer learning model in PyTorch by finetuning ResNet18.

## MEL baseline model v0

The MEL (Manually Expert Labeled) database contains original annotations of the three expert pathologists who went through the 21 WSIs. The other two databases ODAEL and CODAEL used object detection and object detection plus clustering to 'boost' the number of training examples, respectively. So training a model on the MEL data represents a good baseline or starting point for the cell classifier.

The [model_baseline_MEL_v0](https://github.com/artemiorimando/samsung-capstone/blob/master/assignments/week-8-iteration/baseline_models/model_baseline_MEL_v0.ipynb) notebook has all the code necessary to train and evaluate the model. The notebook was run on Google Colab Pro with GPUs, which greatly reduced training time vs. local CPUs.

### Transfer learning model

The PyTorch instance of ResNet18 was loaded and the last fully connected layer was reset for binary classification. The model was set to 'finetune' mode where all the weights and biases in the network are updated during training. The 'feature extract' mode in constrast freezes the weights of the network and only updates the weights of the last fully connected layer.

### Key hyperparameters

* batch size = 64
* number of workers = 2
* optimizer/loss function = stochastic gradient descent
* learning rate initial = 0.01
* learning rate scheduler = StepLR(step size = 7, gamma = 0.1)
* momentum = 0.9
* epochs = 25

### Data transforms

The dataset used was MEL_1: MEL database and the first test slide split.

Since the data was loaded in with PyTorch ImageFolder, it created two datasets: train and test based on the directory structure discussed earlier. The train data was randomly split 80/20 to create a validation set. The mean and standard deviation of the training set after splitting was calculated for normalization.
* mean = [0.7438, 0.5271, 0.6297]
* std = [0.1433, 0.1634, 0.1339]

The training, validation, and test sets were all resized to the ResNet18 required input size of 224 x 224 pixels. Images in the training set were flipped on the horizontal with equal (50%) probability. The mean and std of the training set was used to normalize the validation and test sets.

### Model results

The 'best' model as determined from highest validation accuracy was from epoch 9 out of 25. On subsequent epochs, the train accuracy increased while the validation accuracy remained relatively constant, perhaps suggesting overfitting. The runtime was 51m 35s.
* train Loss: 0.0068 Acc: 0.9990
* val Loss: 0.6853 Acc: 0.8536

Classification report

|              | precision | recall   | f1-score | support |
|--------------|-----------|----------|----------|---------|
| Mitosis      | 0.76      | 0.79     | 0.78     | 3292    |
| Nonmitosis   | 0.90      | 0.89     | 0.89     | 7036    |
|              |           | accuracy | 0.85     | 10328   |
| macro avg    | 0.83      | 0.84     | 0.83     | 10328   |
| weighted avg | 0.86      | 0.85     | 0.86     | 10328   |

Confusion matrix

![cm](./img/MEL_v0_cm.png)

### MEL v0 results discussion

In general, the model performs decently, but the class imbalance of the training data is evident in the difference between precision and recall values for mitosis and nonmitosis. The cell patch classifier is good at identifying nonmitosis but at the expense of correctly identifying mitosis.

## CODAEL baseline model v0

The CODAEL (Clustering and object detection expert labeled) dataset is arguably the most refined dataset. The author's used a series of CNNs to identify image patch candidates that the expert pathologists potentially missed and had the pathologists re-evaluate these patch candidates. This technique increased the number of training examples for mitosis and nonmitosis.

### Transfer learning model

[model_baseline_CODAEL_v0.ipynb](https://github.com/artemiorimando/samsung-capstone/blob/master/assignments/week-8-iteration/baseline_models/model_baseline_CODAEL_v0.ipynb) contains the necessary code to run the CODAEL v0 model. The notebook was run on Google Colab Pro with GPUs.

The same model (MEL v0), hyperparameters, and data transforms described above were used for training on the CODAEL_1 dataset. The mean and std of the training set were recalculated for normalization.
* mean = [0.7508, 0.5410, 0.6403]
* std = [0.1428, 0.1637, 0.1340]

### Model results

The 'best' model was from the last epoch (25). The runtime was 64m 51s. The validation accuracy stayed relatively constant after ~5 epochs while the training accuracy increased to ~100% by epoch 25. This behavior suggests overfitting.
* train Loss: 0.00068 Acc: ~1
* val Loss: 0.750 Acc: 0.878

Classification report

|              | precision | recall   | f1-score | support |
|--------------|-----------|----------|----------|---------|
| Mitosis      | 0.80      | 0.72     | 0.76     | 3412    |
| Nonmitosis   | 0.90      | 0.93     | 0.91     | 8920    |
|              |           | accuracy | 0.87     | 12332   |
| macro avg    | 0.85      | 0.83     | 0.84     | 12332   |
| weighted avg | 0.87      | 0.87     | 0.87     | 12332   |

Confusion matrix

![cm](./img/CODAEL_v0_cm.png)

### CODAEL v0 results discussion

Like the MEL v0 model, the CODAEL model identifies nonmitosis very well, but struggles with correctly classifying mitosis. The mitosis precision slightly increased but at the expense of recall.

## Learnings/future work

### Model performance

The class imbalance in the training data clearly affects the performance of the model. Focus areas for improvement in the next iteration are listed below in order of importance (what we think is the highest to gain to effort ratio)
1. Data-centric approach
    * Leverage minority class oversampling, majority class undersampling, and/or other sampling technique to address class imbalance
    * Perform more data augmentations to reduce overfitting
    * Use smaller amount of training data to see if comparable results can be achieved. If so, this could help reduce training run time
2. Model/hyperparameter tuning
    * Construct ROC curves to inform choice of classification threshold to maximize metric of interest
    * Reduce learning rate and number of epochs to address overfitting

### Model evaluation and tracking

PyTorch unfortunately does not support MLflow autologging (however PyTorch lightning does) and thus requires additional code setup. Since we want to stay with PyTorch and move from Google Colab to AWS Sagemaker, we will attempt to implement MLflow with PyTorch in Sagemaker on the next model iteration. 

ROC and AUC will be important metrics for optimizing the classification threshold of the next model iteration.

### Patch candidate/object detection model

The cell patch classification model solves how to identify mitosis vs. nonmitosis on image patches. However, we need to be able to generate these patches from WSIs that have NOT been annotated. The author's have a FastAI implementation of a RetinaNet based model that randomly samples crops from WSIs and generates patch candidates, which are the inputs to the cell patch classifier; the result is a 'two-stage' model. A key next step on the modeling side is replicating this in PyTorch.

### Thinking about deployment

Regardless of how well we can get the models to perform, arguably the more important aspect is having a template/framework for deploying these models along with the explainability features. So though there is relevant work to be done improving model performance, we will be focusing on deployment and the modularity of the pipeline such that if a great performing model is trained down the road, we can easily deploy it.